### Basic Setup
Here we check the sage version, configure some settings, define a 2-manifold $M$ with cylidrical coordinate chart $Y$, and a riemannian metric. The naming convention python variables (not variables used in the actual expressions) is `CamelCase` for objects defined on the manifold, and `snake_case` (with a trailing underscore for functions) for generic symbolic expressions. For rank $(0, 2)$ tensors, we default to the purely covariant form (as that is what we have regularization conditions for), denote the $(1, 1)$ version as `X_mat` and denote the rank $(2, 0)$ version as `X_con`.

In [1]:
# Check version
version()

'SageMath version 9.5, Release Date: 2022-01-30'

In [2]:
# Reset
reset()
# Setup pretty printing
%display latex

In [3]:
# Define a differentiable manifold of dimension 2 over real numbers
M = Manifold(2, 'M', latex_name=r'\mathcal{M}', start_index=1)
# Define a cylindrical chart on the manifold
Chart.<rho, z> = M.chart(r'rho:\rho z:z')
# Cache frame
Chartf = Chart.frame()

In [4]:
# Declare a metric on the manifold
G = M.riemannian_metric('g')

# Components of Metric
grr_ = function('grr_f', latex_name='g_{rr}')(rho, z)
gzz_ = function('gzz_f', latex_name='g_{zz}')(rho, z)
grz_ = function('grz_f', latex_name='g_{rz}')(rho, z)

# Metric is symmetric
G[1, 1], G[2, 2] = grr_, gzz_
G[1, 2] = grz_

# Invert metric
G_con = G.inverse()

# Print metric
G.display()

g = grr_f(rho, z) drho⊗drho + grz_f(rho, z) drho⊗dz + grz_f(rho, z) dz⊗drho + gzz_f(rho, z) dz⊗dz

In [7]:
G_con.display()

inv_g = -gzz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂rho⊗∂/∂rho + grz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂rho⊗∂/∂z + grz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂z⊗∂/∂rho - grr_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂z⊗∂/∂z

In [16]:
# Compute Ricci Tensor
R = G.ricci()
# And Ricci Scalar
R_trace = G_con['^ij'] * R['_ij']
# And connection coefficients, used for covariant derivative
Nabla = G.connection(name='nabla', latex_name=r'\nabla')
# Make sure metric is compatible
assert(Nabla(G) == 0)

Nabla.display()

Gam^rho_rho,rho = -1/2*(gzz_f(rho, z)*d(grr_f)/drho + grz_f(rho, z)*d(grr_f)/dz - 2*grz_f(rho, z)*d(grz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_rho,z = -1/2*(gzz_f(rho, z)*d(grr_f)/dz - grz_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_z,rho = -1/2*(gzz_f(rho, z)*d(grr_f)/dz - grz_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_z,z = -1/2*(2*gzz_f(rho, z)*d(grz_f)/dz - gzz_f(rho, z)*d(gzz_f)/drho - grz_f(rho, z)*d(gzz_f)/dz)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_rho,rho = 1/2*(grz_f(rho, z)*d(grr_f)/drho + grr_f(rho, z)*d(grr_f)/dz - 2*grr_f(rho, z)*d(grz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_rho,z = 1/2*(grz_f(rho, z)*d(grr_f)/dz - grr_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_z,rho = 1/2*(grz_f(rho, z)*d(grr_f)/dz - grr_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_z,z = 1/2*(2*grz_f(rho, z)*d(grz_f)/dz - grz_f(rho, z)*d(gzz_f)/drho - grr_f(rho, z)*d(gzz_f)/dz)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z))

### Variables
Next we define the relavent tensor fields, as well as their components in our default frame.

In [6]:
s_t_ = function('seed_f', latex_name='s')(rho, z)

LamT = M.scalar_field(rho * exp(rho * s_t_) * sqrt(grr_), chart=Chart, name="lam_t", latex_name=r'\lambda')

LamLogT = Nabla(LamT) / LamT
LamLog2T = Nabla(LamLogT) + LamLogT * LamLogT

LamLog2T[1, 1].expr().simplify_full()

-1/4*(rho*grr_f(rho, z)^2*diff(grr_f(rho, z), z)^2 - 4*(rho*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho*grr_f(rho, z)^3*gzz_f(rho, z))*seed_f(rho, z)^2 + (rho*grz_f(rho, z)^2 - 2*rho*grr_f(rho, z)*gzz_f(rho, z))*diff(grr_f(rho, z), rho)^2 - 4*(rho^3*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho^3*grr_f(rho, z)^3*gzz_f(rho, z))*diff(seed_f(rho, z), rho)^2 - 8*(grr_f(rho, z)^2*grz_f(rho, z)^2 - grr_f(rho, z)^3*gzz_f(rho, z))*seed_f(rho, z) - 2*(2*grr_f(rho, z)*grz_f(rho, z)^2 - grr_f(rho, z)^2*gzz_f(rho, z) + (2*rho*grr_f(rho, z)*grz_f(rho, z)^2 - rho*grr_f(rho, z)^2*gzz_f(rho, z))*seed_f(rho, z))*diff(grr_f(rho, z), rho) - 2*(rho*grr_f(rho, z)*grz_f(rho, z)^2 - rho*grr_f(rho, z)^2*gzz_f(rho, z))*diff(grr_f(rho, z), rho, rho) - 2*(rho*grr_f(rho, z)^2*grz_f(rho, z)*seed_f(rho, z) + grr_f(rho, z)^2*grz_f(rho, z))*diff(grr_f(rho, z), z) + 2*(2*rho*grr_f(rho, z)^2*grz_f(rho, z)*seed_f(rho, z) + rho*grr_f(rho, z)*grz_f(rho, z)*diff(grr_f(rho, z), rho) - rho*grr_f(rho, z)^2*diff(grr_f(rho, z), z) + 2*grr_f(rho, z)^2*grz_f(rho, z))*diff(grz_f(rho, z), rho) - 2*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)*diff(grr_f(rho, z), z) - 2*rho^2*grr_f(rho, z)^2*grz_f(rho, z)*diff(grz_f(rho, z), rho) + 8*rho*grr_f(rho, z)^2*grz_f(rho, z)^2 - 8*rho*grr_f(rho, z)^3*gzz_f(rho, z) + 4*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho^2*grr_f(rho, z)^3*gzz_f(rho, z))*seed_f(rho, z) + (2*rho^2*grr_f(rho, z)*grz_f(rho, z)^2 - rho^2*grr_f(rho, z)^2*gzz_f(rho, z))*diff(grr_f(rho, z), rho))*diff(seed_f(rho, z), rho) - 4*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho^2*grr_f(rho, z)^3*gzz_f(rho, z))*diff(seed_f(rho, z), rho, rho) + 2*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)*diff(grr_f(rho, z), rho) + rho^2*grr_f(rho, z)^3*diff(grr_f(rho, z), z) - 2*rho^2*grr_f(rho, z)^3*diff(grz_f(rho, z), rho))*diff(seed_f(rho, z), z))/(rho*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho*grr_f(rho, z)^3*gzz_f(rho, z))

In [18]:
(Nabla(Nabla(LamT)) / LamT)[1, 2].expr().subs({grz_: 0, diff(grr_, rho): 0, diff(gzz_, rho): 0}).expand()

rho^2*diff(seed_f(rho, z), rho)*diff(seed_f(rho, z), z) + rho*seed_f(rho, z)*diff(seed_f(rho, z), z) + rho*diff(seed_f(rho, z), rho, z) + 1/2*diff(grr_f(rho, z), rho, z)/grr_f(rho, z) + 2*diff(seed_f(rho, z), z)

In [30]:
# Phi component of metric
lam_ = function('lam_f', latex_name=r'\lambda')(rho, z)
# Logrithmic derivatives of lambda
# These terms are not regular on axis, and thus must be replaced with more specific free variables
# (ie Lambda_a Z^a).
lam_logr_ = function('lam_logr_f', latex_name=r'\Lambda_r')(rho, z)
lam_logz_ = function('lam_logz_f', latex_name=r'\Lambda_z')(rho, z)

# Resulting Tensors
Lam = M.scalar_field(lam_, chart=Chart, name='lam_s', latex_name=r'\lambda')
          
Lam.display()

lam_s: M → ℝ
   (rho, z) ↦ lam_f(rho, z)

In [29]:
# Instead we use free varaibles so we can later substitute this for a regular term.

# Logrithmic second derivatives of lambda
# These derivatives are indeed regular
lam_log2rr_ = function('lam_log2rr_f', latex_name=r'\lambda^{-1} (\nabla_r \nabla_r \lambda)')(rho, z)
lam_log2zz_ = function('lam_log2zz_f', latex_name=r'\lambda^{-1} (\nabla_z \nabla_z \lambda)')(rho, z)
lam_log2rz_ = function('lam_log2rz_f', latex_name=r'\lambda^{-1} (\nabla_r \nabla_z \lambda)')(rho, z)

# Second Logrithmic Derivative of lambda, equal to `Nabla(Nabla(Lam)) / Lam`
LamLog2 = M.tensor_field(0, 2, sym=(0, 1), name="LamLog2", latex_name=r'\lambda^{-1} (\nabla \nabla \lambda)')
LamLog2.add_comp(Chartf)[1, 1] = lam_log2rr_
LamLog2.add_comp(Chartf)[1, 2] = lam_log2rz_
LamLog2.add_comp(Chartf)[2, 2] = lam_log2zz_

LamLog2.display()

LamLog2 = lam_log2rr_f(rho, z) drho⊗drho + lam_log2rz_f(rho, z) drho⊗dz + lam_log2rz_f(rho, z) dz⊗drho + lam_log2zz_f(rho, z) dz⊗dz

In [33]:
# Gauge Fields
lapse_ = function('lapse_f', latex_name=r'\alpha')(rho, z)

shiftr_ = function('shiftr_f', latex_name=r'\beta^r')(rho, z)
shiftz_ = function('shiftz_f', latex_name=r'\beta^z')(rho, z)

# Gauge Tensors
Lapse = M.scalar_field(lapse_, chart=Chart, name='Lapse', latex_name=r'\alpha')

Shift = M.tensor_field(1, 0, name='Shift', latex_name=r'\beta')
Shift.add_comp(Chartf)[1] = shiftr_
Shift.add_comp(Chartf)[2] = shiftz_

Lapse.display()

Lapse: M → ℝ
   (rho, z) ↦ lapse_f(rho, z)

In [34]:
Shift.display()

Shift = shiftr_f(rho, z) ∂/∂rho + shiftz_f(rho, z) ∂/∂z

In [39]:
# Extrinsic Curvature

# Components
krr_ = function('Krr_f', latex_name='K_{rr}')(rho, z)
kzz_ = function('Kzz_f', latex_name='K_{zz}')(rho, z)
krz_ = function('Krz_f', latex_name='K_{rz}')(rho, z)

# Phi Component of K
# y_ = function('Y')(rho, z)
# l_ = k_rr_/g_rr_ + rho * y_
l_ = function('L_f', latex_name='L')(rho, z)

K = M.tensor_field(0, 2, sym=(0, 1), name='K')
K.add_comp(Chartf)[1, 1] = krr_
K.add_comp(Chartf)[2, 2] = kzz_
K.add_comp(Chartf)[1, 2] = krz_

K_mat = G_con['^{ij}'] * K['_jk']
K_con = K_mat['^i_k'] * G_con['^{kj}']
K_trace = G_con['^ij'] * K['_ij']

# L = K_\phi^\phi
L = M.scalar_field(l_, name='L', latex_name=r'L')

lam_kr_ = function('lam_kr_f', latex_name=r'\Lambda')

K.display()

K = Krr_f(rho, z) drho⊗drho + Krz_f(rho, z) drho⊗dz + Krz_f(rho, z) dz⊗drho + Kzz_f(rho, z) dz⊗dz

In [128]:
L.display()

L: M → ℝ
   (rho, z) ↦ L_f(rho, z)

In [129]:
# Z4
theta_ = function('theta_f', latex_name=r'\theta')(rho, z)
Theta = M.scalar_field(theta_, chart=Chart, name='theta_s', latex_name=r'\theta')

zr_ = function('Zr_f', latex_name=r'Z_r')(rho, z)
zz_ = function('Zz_f', latex_name=r'Z_z')(rho, z)

Z = M.tensor_field(0, 1, name='Z')
Z.add_comp(Chartf)[1] = zr_
Z.add_comp(Chartf)[2] = zz_

# Vector form of Z
Zv = G_con['^ij'] * Z['_j']

Theta.display()

theta_s: M → ℝ
   (rho, z) ↦ theta_f(rho, z)

In [130]:
Z.display()

Z = Zr_f(rho, z) drho + Zz_f(rho, z) dz

### Regularization

We want the following equations to be well-defined for both $r > 0$ and $r = 0$. Unfourtunately, to numerically ensure regularity, we must substitute $\lambda \to r e^{r s} \sqrt{g_{rr}}$ and $L = K_{rr} / g_{rr} + rY$. Let 
$$ \Lambda_a = \lambda^{-1} \nabla_a \lambda $$
$$ A_a = \alpha^{-1} \nabla_a \alpha. $$ 
The term $\Lambda_a$ shows up at various points throughout the equations, and is clearly $\mathcal{O}(r^{-1})$ on axis. Taking into account the on-axis behaviour of the other variables, these terms do get cancelled out properly, but Sage does not know this on-axis behaviour (and thus cannot automatically make simplifications like $s/r \to \partial_r s$ when $r = 0$). In order to get around this we define several free variables, and manually make the correct substitution depending on whether $r = 0$.
$$ N = \Lambda_a Z^a $$
$$ O = \Lambda_a A^a $$
$$ P_a = \Lambda_b K_{a}^{\;b}  - \Lambda_a L $$
$$ Q_{ab} = \lambda^{-1} \nabla_a \nabla_b \lambda = \nabla_a \Lambda_b + \Lambda_a \Lambda_b $$

In [37]:
n_ = function('n_f', latex_name='N')(rho, z)
o_ = function('o_f', latex_name='O')(rho, z)

N = M.scalar_field(n_, chart=Chart, name='N')
O = M.scalar_field(o_, chart=Chart, name='O')

pr_ = function('pr_f', latex_name="P_r")(rho, z)
pz_ = function('pz_f', latex_name="P_z")(rho, z)

P = M.tensor_field(0, 1, sym=(0, 1), name='P')
P.add_comp(Chartf)[1] = pr_
P.add_comp(Chartf)[2] = pz_

qrr_ = function('qrr_f', latex_name='Q_{rr}')(rho, z)
qrz_ = function('qrz_f', latex_name='Q_{rz}')(rho, z)
qzz_ = function('qzz_f', latex_name='Q_{zz}')(rho, z)

Q = M.tensor_field(0, 2, sym=(0, 1), name='Q')
Q.add_comp(Chartf)[1, 1] = qrr_
Q.add_comp(Chartf)[2, 2] = qzz_
Q.add_comp(Chartf)[1, 2] = qrz_

### Constraint equations
Constraint equations that should be satisfied at all times.
$$ \mathcal{C} \equiv \frac{1}{2} (K^2 - K_{ij} K^{ij} + R) - \lambda^{-1} \nabla^j \nabla_j \lambda  + K L = 0 $$

$$ \mathcal{C}_i \equiv \nabla_j K_{i}^{\;j} - \nabla_i (K + L) + \lambda^{-1} (\nabla_j \lambda) K_{i}^{\;j} - \lambda^{-1} (\nabla_i \lambda) L = 0$$

In [43]:
term1 = (K_trace^2 - K['_ij'] * K_con['^ij'] + R_trace) / 2
term2 = - Q['_ij'] * G_con['^{ij}'] + K_trace * L

# Hamiltonian
CH = term1 + term2

CH

Scalar field on the 2-dimensional differentiable manifold M

In [47]:
# term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
# term2 = LamLog['_j'] * K_mat['^j_i'] - LamLog * L

# Momentum Constraint
CM = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L) + P

CM

1-form on the 2-dimensional differentiable manifold M

### Evolution Equations
Calculations for $\mathcal{L}_n X$ for various dynamical quantities.

In [135]:
term1 = R - Q - Nabla(Nabla(Lapse)) / Lapse
term2 = (K_trace + L) * K - 2 * K['_ij'] * K_mat['^j_k']
term3 = 2 * Nabla(Z)['_(ij)'] - 2 * K * Theta

# Extrinsic Curvature
LieK = term1 + term2 + term3

In [106]:
term1 = - Q['_ij'] * G_con['^{ij}'] - O
term2 = L * (K_trace + L) + 2 * N - 2 * L * Theta

# Angular Extrinsic Curvature
LieL = term1 + term2

In [107]:
# Metric and Lambda
LieG = -2*K
LieLam = -Lam * L

In [108]:
term1 = CH + N - Nabla(Lapse)['_i'] * Zv['^i'] / Lapse 
term2 = Nabla(Zv)['^i_i'] - (K_trace + L) * Theta

# Theta
LieTheta = term1 + term2

In [109]:
term1 = CM - 2 * K['_ij'] * Zv['^j']
term2 = - Nabla(Lapse) / Lapse * Theta + Nabla(Theta) 

# Z
LieZ = term1 + term2

In [110]:
# Actual temporal derivatives including shift terms
G_t = Lapse * LieG + G.lie_derivative(Shift)
K_t = Lapse * LieK + K.lie_derivative(Shift)

Theta_t = Lapse * LieTheta + Theta.lie_derivative(Shift)
Z_t = Lapse * LieZ + Z.lie_derivative(Shift)

### Preprocessing
To avoid transcription errors when transferring between notebooks and code we automatically transform symbolic expressions into `C` code.

In [113]:
import sympy as sp

# Symbolic Variables to replace current functions
var('grr_r grr_z grr_rr grr_zz grr_rz grr')
var('gzz_r gzz_z gzz_rr gzz_zz gzz_rz gzz')
var('grz_r grz_z grz_rr grz_zz grz_rz grz')
# var('s_r s_z s_rr s_zz s_rz s')
var('lam_r lam_z lam_rr lam_zz lam_rz lam')
var('lam_logr lam_logz')

var('krr_r krr_z krr')
var('kzz_r kzz_z kzz')
var('krz_r krz_z krz')
# var('Y_r Y_z Y')
var('l_r l_z l')

var('lapse_r lapse_z lapse_rr lapse_zz lapse_rz lapse')
var('shiftr_r shiftr_z shiftz_r shiftz_z shiftr shiftz')

var('theta_r theta_z theta')
var('zr_r zr_z zz_r zz_z zr zz')

"""
Preprocesses an expression, replacing all derivatives of a function
and invokations of that functions with an appropriately named variable.
"""
def process_expr(expr):
    expr = expr.simplify_full()
    derives = expr.subs({
        diff(grr_, rho): grr_r,
        diff(grr_, z): grr_z,
        diff(grr_, rho, rho): grr_rr,
        diff(grr_, z, z): grr_zz,
        diff(grr_, rho, z): grr_rz,
        diff(grr_, z, rho): grr_rz,
        
        diff(gzz_, rho): gzz_r,
        diff(gzz_, z): gzz_z,
        diff(gzz_, rho, rho): gzz_rr,
        diff(gzz_, z, z): gzz_zz,
        diff(gzz_, rho, z): gzz_rz,
        diff(gzz_, z, rho): gzz_rz,
        
        diff(grz_, rho): grz_r,
        diff(grz_, z): grz_z,
        diff(grz_, rho, rho): grz_rr,
        diff(grz_, z, z): grz_zz,
        diff(grz_, rho, z): grz_rz,
        diff(grz_, z, rho): grz_rz,
        
        diff(krr_, rho): krr_r,
        diff(krr_, z): krr_z,
        
        diff(kzz_, rho): kzz_r,
        diff(kzz_, z): kzz_z,
        
        diff(krz_, rho): krz_r,
        diff(krz_, z): krz_z,
        
#         diff(s_, rho): s_r,
#         diff(s_, z): s_z,
#         diff(s_, rho, rho): s_rr,
#         diff(s_, z, z): s_zz,
#         diff(s_, rho, z): s_rz,
#         diff(s_, z, rho): s_rz,
        
        diff(lam_, rho): lam_r,
        diff(lam_, z): lam_z,
        diff(lam_, rho, rho): lam_rr,
        diff(lam_, z, z): lam_zz,
        diff(lam_, rho, z): lam_rz,
        diff(lam_, z, rho): lam_rz,
        
        diff(lapse_, rho): lapse_r,
        diff(lapse_, z): lapse_z,
        diff(lapse_, rho, rho): lapse_rr,
        diff(lapse_, z, z): lapse_zz,
        diff(lapse_, rho, z): lapse_rz,
        diff(lapse_, z, rho): lapse_rz,
        
        diff(shiftr_, rho): shiftr_r,
        diff(shiftr_, z): shiftr_z,
        diff(shiftz_, rho): shiftz_r,
        diff(shiftz_, z): shiftz_z,
        
#         diff(y_, rho): Y_r,
#         diff(y_, z): Y_z,
        
        diff(l_, rho): l_r,
        diff(l_, z): l_z,
        
        diff(theta_, rho): theta_r,
        diff(theta_, z): theta_z,
        
        diff(zr_, rho): zr_r,
        diff(zr_, z): zr_z,
        diff(zz_, rho): zz_r,
        diff(zz_, z): zz_z,
    })
    
    values = derives.subs({
        grr_: grr,
        gzz_: gzz,
        grz_: grz,
#         s_: s,
        lam_: lam,
        lam_logr_: lam_logr,
        lam_logz_: lam_logz,
        
        krr_: krr,
        kzz_: kzz,
        krz_: krz,
#         y_: Y,
        l_: l,
        
        lapse_: lapse,
        shiftr_: shiftr,
        shiftz_: shiftz,
        
        theta_: theta,
        zr_: zr,
        zz_: zz,
    })
    
    return values

In [114]:
def generate_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(process_expr(eq)._sympy_().simplify())
        
    subs, final = sp.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sp.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        result += "double " + str(name) + " = " + str(expr) + ";\n\n"
        
    return result

hyperbolic = [
    ("grr_t", G_t[1, 1].expr()),
    ("gzz_t", G_t[2, 2].expr()),
    ("grz_t", G_t[1, 2].expr()),
    ("Krr_t", K_t[1, 1].expr()),
    ("Kzz_t", K_t[2, 2].expr()),
    ("Krz_t", K_t[1, 2].expr()),
    ("theta_t", Theta_t.expr()),
    ("Zr_t", Z_t[1].expr()),
    ("Zz_t", Z_t[2].expr()),
]

# hyperbolic_h = open("hyperbolic.h", 'w')
# hyperbolic_h.write(generate_ccode(hyperbolic))
# hyperbolic_h.close()

print(generate_ccode(hyperbolic))

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = 2*grr;
double x1 = 2*grz;
double x2 = 2*lapse;
double x3 = krr*x2;
double x4 = 2*gzz;
double x5 = kzz*x2;
double x6 = krz*x2;
double x7 = pow(grz, 4);
double x8 = pow(grr, 2);
double x9 = pow(gzz, 2);
double x10 = x8*x9;
double x11 = grr*gzz;
double x12 = pow(grz, 2);
double x13 = 2*x12;
double x14 = x11*x13;
double x15 = x10 - x14 + x7;
double x16 = 1.0/x15;
double x17 = (1.0/4.0)*lapse;
double x18 = pow(grr_z, 2)*x17;
double x19 = pow(gzz_r, 2)*x17;
double x20 = x11 - x12;
double x21 = grr*x20;
double x22 = grz_rz*lapse;
double x23 = (1.0/2.0)*lapse;
double x24 = x21*x23;
double x25 = grr_z*grz;
double x26 = grr_r*gzz;
double x27 = grz_r*x1;
double x28 = x26 - x27;
double x29 = x25 + x28;
double x30 = grz_z*x23;
double x31 = x29*x30;
double x32 = grr*gzz_z;
double x33 = grr_r*grz;
double x34 = grr*grr_z - grz_r*x0